In [8]:
import numpy as np
import random
from glob import glob

#We're using function from image_recognition to process photos
# from image_recognition import

#This is where the classification and categorization of images will occur
data_dir = 'Dataset'
train = 'Dataset/training'
test = 'Dataset/evaluation'
valid = 'Dataset/validation'
label_names = ['Bread', 'Dairy Product', 'Dessert', 'Egg', 'Fried Food', 'Meat', 'Noodles-Pasta', 'Rice',
               'Seafood', 'Soup', 'Vegetable-Fruit']

training_files = glob('Dataset/training/*/*.jpg')
training_set_size = [50,100, 250, 500, 1000, 5000, 9000]

new_training_set = []


This is element 0: Dataset/training\Noodles-Pasta\157.jpg: <class 'str'>
This is element 1: Dataset/training\Bread\556.jpg: <class 'str'>
This is element 2: Dataset/training\Rice\90.jpg: <class 'str'>
This is element 3: Dataset/training\Meat\418.jpg: <class 'str'>
This is element 4: Dataset/training\Meat\430.jpg: <class 'str'>
This is element 5: Dataset/training\Bread\960.jpg: <class 'str'>
This is element 6: Dataset/training\Vegetable-Fruit\240.jpg: <class 'str'>
This is element 7: Dataset/training\Soup\898.jpg: <class 'str'>
This is element 8: Dataset/training\Dessert\67.jpg: <class 'str'>
This is element 9: Dataset/training\Dessert\1468.jpg: <class 'str'>
This is element 10: Dataset/training\Soup\5.jpg: <class 'str'>
This is element 11: Dataset/training\Noodles-Pasta\21.jpg: <class 'str'>
This is element 12: Dataset/training\Soup\275.jpg: <class 'str'>
This is element 13: Dataset/training\Vegetable-Fruit\411.jpg: <class 'str'>
This is element 14: Dataset/training\Vegetable-Fruit\463

In [38]:
import os
# tensorflow
import tensorflow.keras as keras
import tensorflow as tf

# image processing
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img

# model / neural network
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

datagen = ImageDataGenerator()
def gen_training_data(size=10):
    total_num_training_images = 9866.
    return keras.utils.image_dataset_from_directory( 
        directory=train, 
        image_size=(224, 224),
        shuffle = True,
        subset = "training",
        validation_split = 1-(size/total_num_training_images),
        batch_size=50, 
        seed = 512365
    )
# validation data
valid_generator = datagen.flow_from_directory( 
    directory=valid, 
    classes = label_names,
    target_size=(224, 224), 
    batch_size=50, 
    class_mode="binary", 
    shuffle = True
)

    # build the entire model
    # Code taken from 
    # https://github.com/ovh/ai-training-examples/blob/main/notebooks/computer-vision/image-classification/tensorflow/resnet50/notebook-resnet-transfer-learning-image-classification.ipynb
    # Created by eleapttn
def model_creation():
    resnet_50 = ResNet50(include_top=False, weights='imagenet', input_shape=(224,224,3))
    for layer in resnet_50.layers:
        layer.trainable = False
    x = resnet_50.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(512, activation='relu')(x) 
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(256, activation='relu')(x) 
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(128, activation='relu')(x) 
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(64, activation='relu')(x) 
    x = layers.Dropout(0.5)(x)
    predictions = layers.Dense(11, activation='softmax')(x)
    return (Model(inputs = resnet_50.input, outputs = predictions))

# define training function
    # Code taken from 
    # https://github.com/ovh/ai-training-examples/blob/main/notebooks/computer-vision/image-classification/tensorflow/resnet50/notebook-resnet-transfer-learning-image-classification.ipynb
    # Created by eleapttn
def trainModel(model, passes, optimizer, training_data):
    batch_size = 32
    model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model.fit(training_data, validation_data=valid_generator, epochs=passes, batch_size=batch_size)

Found 3430 images belonging to 11 classes.


In [39]:
test_data = datagen.flow_from_directory( 
    directory=test, 
    classes = label_names,
    target_size=(224, 224), 
    batch_size=32, 
    class_mode="binary",
    shuffle=False
)


def find_accuracy(labels, predictions):
    sum = 0
    for i in range(len(labels)):
        if labels[i]==predictions[i]:
            sum +=1
    return sum/len(labels)


test_class = test_data.labels
training_Accuracy = []
validation_Accuracy = []
testing_Accuracy = []
for size in training_set_size:
    print(size)
    training_data = gen_training_data(size)
    model = model_creation()
    model_history = trainModel(model, passes=10, optimizer="Adam", training_data=training_data)
    predictions = model.predict(test_data)
    class_ids = np.argmax(predictions, axis=1)
    training_Accuracy.append(model_history)
    validation_Accuracy.append(model_history)
    testing_Accuracy.append(find_accuracy(test_class, class_ids))
    

Found 3347 images belonging to 11 classes.
50
Found 9866 files belonging to 11 classes.
Using 50 files for training.
Epoch 1/10
1/1 [==============================] - ETA: 0s - loss: 4.5518 - accuracy: 0.0200

KeyboardInterrupt: 